## <font color="blue"> Data</font>

In [1]:
using DataFrames
using JWAS.Datasets
using JWAS.MT

In [2]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [3]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [4]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [5]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


### Genetic covariance matrix and residual covariance matrix

In [6]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

## <font color="blue">1. mulitiple trait with fixed effects</font> 

In [7]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


#### set up model equations

In [8]:
model_equations = "BW = intercept + age + sex;
                  CW = intercept + age + sex";

In [9]:
model1 = MT.build_model(model_equations,R);

#### set variables as covariate (all variables are set as factors by default)

In [10]:
MT.set_covariate(model1,"age")

#### run

In [11]:
out1=MT.runMCMC(model1,data,chain_length=50000,printout_frequency=10000);

LoadError: LoadError: UndefVarError: constraint not defined
while loading In[11], in expression starting on line 1

## <font color="blue">2. multiple trait with direct genetic effects</font> 

#### set up model equations

In [12]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [13]:
model2    = MT.build_model(model_equations,R);

In [14]:
MT.set_covariate(model1,"age")

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [15]:
ped = MT.get_pedigree(pedfile);

In [16]:
MT.set_random(model2,"Animal", ped,G)

#### run

In [17]:
out2 = MT.runMCMC(model2,data,chain_length=50000,printout_frequency=10000);

LoadError: LoadError: UndefVarError: constraint not defined
while loading In[17], in expression starting on line 1

## <font color="blue">3. mulitiple traits with marker effects</font> 

#### set up model equations

In [18]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [19]:
model3    = MT.build_model(model_equations,R);

In [20]:
MT.set_covariate(model3,"age")

In [21]:
MT.add_markers(model3,genofile,G,separator=',',header=true);

The delimiters in file /Users/haocheng/.julia/v0.4/JWAS/src/5.Datasets/src/../data/testMT/genotype.txt is ,  .


In [22]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0;0.0]=>0.25,[0.0,1.0]=>0.25,[0.0; 0.0]=>0.25)
out2 = MT.runMCMC(model3,data,Pi=Pi,chain_length=50000,printout_frequency=10000,methods="BayesC");

posterior means at sample: 10000
Residual covariance matrix: 
[76.47588242992278 18.88908178582573
 18.88908178582573 5.383184888923581]
Marker effects covariance matrix: 
[33.60390098636839 3.8358003871628403
 3.8358003871628403 2.494298318682178]

π: 
Dict([0.0,1.0]=>0.21652833048067252,[1.0,0.0]=>0.24695732907041995,[1.0,1.0]=>0.3110189019882298,[0.0,0.0]=>0.22549543846067915)
posterior means at sample: 20000
Residual covariance matrix: 
[83.3811078381825 20.674980149678515
 20.674980149678515 5.837976454669043]
Marker effects covariance matrix: 
[30.581951768935475 4.502643657012362
 4.502643657012362 2.271313218264787]

π: 
Dict([0.0,1.0]=>0.21863485298694946,[1.0,0.0]=>0.24621573776090755,[1.0,1.0]=>0.307486352243359,[0.0,0.0]=>0.2276630570087854)
posterior means at sample: 30000
Residual covariance matrix: 
[89.38787051940122 22.011670817425273
 22.011670817425273 6.136910329345956]
Marker effects covariance matrix: 
[27.071207839784588 3.98337594038868
 3.98337594038868 1.99014

## <font color="blue">4. mulitiple traits with marker effects and polygenic effects</font> 

#### set up model equations

In [23]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [24]:
model4    = MT.build_model(model_equations,R);

In [25]:
MT.set_covariate(model4,"age")

In [26]:
GA = G*0.1
MT.set_random(model4,"Animal",ped,GA)

In [27]:
GM = G*0.9
MT.add_markers(model4,genofile,GM,separator=',',header=true);

The delimiters in file /Users/haocheng/.julia/v0.4/JWAS/src/5.Datasets/src/../data/testMT/genotype.txt is ,  .


#### run

In [28]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0;0.0]=>0.25,[0.0,1.0]=>0.25,[0.0; 0.0]=>0.25)
out4=MT.runMCMC(model4,data,Pi=Pi,chain_length=50000,printout_frequency=10000,methods="BayesC");

posterior means at sample: 10000
Residual covariance matrix: 
[130.01857461779846 32.81659799753262
 32.81659799753262 9.095561238555176]
Marker effects covariance matrix: 
[27.863577925026686 3.702837117847429
 3.702837117847429 1.7429010627545944]

π: 
Dict([0.0,1.0]=>0.22430497628803378,[1.0,0.0]=>0.24224591213714788,[1.0,1.0]=>0.30315833049078944,[0.0,0.0]=>0.23029078108402898)
posterior means at sample: 20000
Residual covariance matrix: 
[121.78602690844771 30.303486692757506
 30.303486692757506 8.337833614877836]
Marker effects covariance matrix: 
[21.465486193213152 2.9245403664018825
 2.9245403664018825 1.458980521449259]

π: 
Dict([0.0,1.0]=>0.22407350145795618,[1.0,0.0]=>0.24642209690521483,[1.0,1.0]=>0.3009316467526234,[0.0,0.0]=>0.22857275488420578)
posterior means at sample: 30000
Residual covariance matrix: 
[109.57927626985605 27.257980923819687
 27.257980923819687 7.560741915252158]
Marker effects covariance matrix: 
[22.364939995385978 3.3325252583956906
 3.33252525839

## <font color="blue">5. check results</font> 

In [29]:
keys(out4)

Base.KeyIterator for a Dict{Any,Any} with 6 entries. Keys:
  "posterior mean of marker effects covariance matrix"
  "posterior mean of residual covaraince matrix"
  "posterior mean of Pi"
  "posterior mean of polygenic effects covariance matrix"
  "posterior mean of marker effects"
  "posterior mean of location parameters"

In [30]:
out4["posterior mean of polygenic effects covariance matrix"]

2x2 Array{Float64,2}:
 2.0712    0.111882
 0.111882  0.194692

In [31]:
out4["posterior mean of location parameters"]

18x2 Array{Any,2}:
 "1:intercept: intercept"  126.496     
 "1:age: age"               -6.10916   
 "1:sex: M"                 30.7993    
 "1:sex: F"                -47.2409    
 "1:Animal: S1"             -0.2025    
 "1:Animal: D1"              0.251537  
 "1:Animal: O1"              0.103791  
 "1:Animal: O3"             -0.109212  
 "1:Animal: O2"              0.0370225 
 "2:intercept: intercept"   39.7416    
 "2:age: age"                0.344531  
 "2:sex: M"                -30.4183    
 "2:sex: F"                -32.5468    
 "2:Animal: S1"             -0.045055  
 "2:Animal: D1"              0.0361508 
 "2:Animal: O1"              0.0116877 
 "2:Animal: O3"             -0.0306742 
 "2:Animal: O2"             -0.00380884

In [32]:
out4["posterior mean of marker effects"][1]

5x2 Array{Any,2}:
 "x1"   3.2871  
 "x2"   1.84795 
 "x3"  -0.825319
 "x4"   1.44957 
 "x5"  -0.624471

In [33]:
out4["posterior mean of marker effects"][2]

5x2 Array{Any,2}:
 "x1"   0.671408
 "x2"   0.386237
 "x3"  -0.186109
 "x4"   0.280759
 "x5"  -0.135505

## <font color="blue">Bonus: maternal effects</font> 

In [34]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [35]:
d2 = [data[3:4,:]  DataFrame(Dam = ["D1","D1"])]

,Animal,BW,CW,age,sex,Dam
1,O1,150.0,13.0,3,M,D1
2,O3,40.0,5.0,4,F,D1


In [36]:
model_equations = "BW = intercept + age + sex + Animal+ Dam;
                   CW = intercept + age + sex + Animal";

In [37]:
model5 = MT.build_model(model_equations,R);

In [38]:
MT.set_covariate(model1,"age")

In [39]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
MT.set_random(model5,"Animal Dam", ped,G0)

In [ ]:
out5=MT.runMCMC(model5,d2,chain_length=50000,printout_frequency=10000);